In [1]:
import feedWebGL2.feedback as fd
import numpy as np
fd.widen_notebook()
np.set_printoptions(precision=4)

In [2]:
n = 40
m = 50
k = 30

M1 = np.arange(n*m).reshape(n,m) * 0.1 + 0.01
M1

array([[1.0000e-02, 1.1000e-01, 2.1000e-01, ..., 4.7100e+00, 4.8100e+00,
        4.9100e+00],
       [5.0100e+00, 5.1100e+00, 5.2100e+00, ..., 9.7100e+00, 9.8100e+00,
        9.9100e+00],
       [1.0010e+01, 1.0110e+01, 1.0210e+01, ..., 1.4710e+01, 1.4810e+01,
        1.4910e+01],
       ...,
       [1.8501e+02, 1.8511e+02, 1.8521e+02, ..., 1.8971e+02, 1.8981e+02,
        1.8991e+02],
       [1.9001e+02, 1.9011e+02, 1.9021e+02, ..., 1.9471e+02, 1.9481e+02,
        1.9491e+02],
       [1.9501e+02, 1.9511e+02, 1.9521e+02, ..., 1.9971e+02, 1.9981e+02,
        1.9991e+02]])

In [3]:
M2 = np.arange(m*k).reshape(m,k) * -10 + 29.1
M2

array([[ 2.9100e+01,  1.9100e+01,  9.1000e+00, ..., -2.4090e+02,
        -2.5090e+02, -2.6090e+02],
       [-2.7090e+02, -2.8090e+02, -2.9090e+02, ..., -5.4090e+02,
        -5.5090e+02, -5.6090e+02],
       [-5.7090e+02, -5.8090e+02, -5.9090e+02, ..., -8.4090e+02,
        -8.5090e+02, -8.6090e+02],
       ...,
       [-1.4071e+04, -1.4081e+04, -1.4091e+04, ..., -1.4341e+04,
        -1.4351e+04, -1.4361e+04],
       [-1.4371e+04, -1.4381e+04, -1.4391e+04, ..., -1.4641e+04,
        -1.4651e+04, -1.4661e+04],
       [-1.4671e+04, -1.4681e+04, -1.4691e+04, ..., -1.4941e+04,
        -1.4951e+04, -1.4961e+04]])

In [4]:
np_product = M1.dot(M2)
np_product

array([[ -1212845.7,  -1214075.7,  -1215305.7, ...,  -1246055.7,
         -1247285.7,  -1248515.7],
       [ -3043070.7,  -3046800.7,  -3050530.7, ...,  -3143780.7,
         -3147510.7,  -3151240.7],
       [ -4873295.7,  -4879525.7,  -4885755.7, ...,  -5041505.7,
         -5047735.7,  -5053965.7],
       ...,
       [-68931170.7, -69024900.7, -69118630.7, ..., -71461880.7,
        -71555610.7, -71649340.7],
       [-70761395.7, -70857625.7, -70953855.7, ..., -73359605.7,
        -73455835.7, -73552065.7],
       [-72591620.7, -72690350.7, -72789080.7, ..., -75257330.7,
        -75356060.7, -75454790.7]])

In [5]:
shader_GLSL_code = """#version 300 es

    // rows are "per mesh"
    in float aRow;
    // columns are "per vertex"
    in float aCol;

    uniform sampler2D RightMatrix;
    uniform sampler2D LeftMatrix;

    // matrix multiplication entry at aRow, aCol
    out float dot_product;

    void main() {
        // foil the optimizer
        gl_Position = vec4(aRow, aCol, aRow, aCol);
        int iCol = gl_VertexID;
        int iRow = gl_InstanceID;
        ivec2 lsize = textureSize(LeftMatrix, 0);
        ivec2 rsize = textureSize(RightMatrix, 0);
        //int N = lsize[0];
        int Klimit = lsize[0];
        // assert Klimit == rsize[1]
        //int M = rsize[1];
        float elt_sum = 0.0;
        for (int k=0; k<Klimit; k++) {
            // data in the red component only.
            //float l_value = texelFetch(LeftMatrix, ivec2(iRow, k), 0).r;
            //float r_value = texelFetch(RightMatrix, ivec2(k, iCol), 0).r;
            float l_value = texelFetch(LeftMatrix, ivec2(k, iRow), 0).r;
            float r_value = texelFetch(RightMatrix, ivec2(iCol, k), 0).r;
            elt_sum += l_value * r_value;
        }
        dot_product = elt_sum;
    }
"""

(n, k) + (M1.shape[0], M2.shape[1])

feedback_program = fd.FeedbackProgram(
    context = fd.Context(
        buffers = fd.Buffers(
            counter_buffer = fd.Buffer(
                array = list(range(max(n, k)))
            ),
        ),
        textures= fd.Textures(
            LeftMatrix= fd.Texture(
                height= M1.shape[0],
                width=  M1.shape[1],
                array= list(M1.ravel()),
            ),
            RightMatrix= fd.Texture(
                height= M2.shape[0],
                width=  M2.shape[1],
                array= list(M2.ravel()),
            ),
        ),
    ),
    runner = fd.Runner(
        num_instances = n,
        vertices_per_instance = k,
        inputs = fd.Inputs(
            aRow = fd.Input(
                num_components = 1,
                from_buffer = fd.BufferLocation(
                    name = "counter_buffer",
                )
            ),
            aCol = fd.Input(
                per_vertex = False,
                num_components = 1,
                from_buffer = fd.BufferLocation(
                    name = "counter_buffer",
                )
            ),
        ),
        samplers = fd.Samplers(
            LeftMatrix = fd.Sampler(
                dim= "2D",
                from_texture= "LeftMatrix",
            ),
            RightMatrix = fd.Sampler(
                dim= "2D",
                from_texture= "RightMatrix",
            ),
        ),
    ),
    program = fd.Program(
        vertex_shader = shader_GLSL_code,
        feedbacks = fd.Feedbacks(
            dot_product = fd.Feedback(num_components=1),
        ),
    ),
)

# display the widget and debugging information
feedback_program.debugging_display()

In [6]:
feedback_program.run()

In [7]:
fb = feedback_program.get_feedback("dot_product")
feedback_product = np.array(fb).reshape((n, k))
feedback_product

array([[ -1212845.75 ,  -1214075.75 ,  -1215305.75 , ...,  -1246055.75 ,
         -1247285.625,  -1248515.75 ],
       [ -3043070.5  ,  -3046800.75 ,  -3050530.75 , ...,  -3143780.75 ,
         -3147511.   ,  -3151240.75 ],
       [ -4873296.   ,  -4879525.5  ,  -4885755.5  , ...,  -5041505.5  ,
         -5047736.   ,  -5053966.   ],
       ...,
       [-68931168.   , -69024904.   , -69118632.   , ..., -71461880.   ,
        -71555624.   , -71649344.   ],
       [-70761400.   , -70857624.   , -70953872.   , ..., -73359608.   ,
        -73455840.   , -73552064.   ],
       [-72591624.   , -72690344.   , -72789064.   , ..., -75257336.   ,
        -75356056.   , -75454792.   ]])

In [8]:
np.allclose(feedback_product, np_product)

True

In [ ]:
%time x = feedback_program.get_feedback("dot_product")

In [ ]:
# %prun x = feedback_program.get_feedback("dot_product")